Unzip

In [1]:
!unzip FleetRL-master.zip

Python 3.10.6


In [ ]:
%cd FleetRL
%pip install --upgrade pip setuptools wheel

In [ ]:
%pip install -r requirements.txt

In [2]:
from FleetRL.fleet_env.fleet_environment import FleetEnv
import time


In [2]:
import os
import time
import gymnasium as gym
import numpy as np
from stable_baselines3 import TD3
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise
import FleetRL

In [5]:
time_now = int(time.time())
comment = "step1"
trained_agents_lmd = f"./trained/last_mile_delivery/TD3-{time_now}-{comment}"
logs_lmd = f"./logs/last_mile_delivery/TD3-{time_now}-{comment}"

if not os.path.exists(trained_agents_lmd):
    os.makedirs(trained_agents_lmd)

if not os.path.exists(logs_lmd):
    os.makedirs(logs_lmd)

No price, static, 36 hours, only soc leave penalty. Best reward possible: 0

In [8]:
env = gym.make('FleetEnv-v0', schedule_name="lmd_sched_single.csv", building_name="load_lmd.csv", include_building=False, include_pv=False, static_time_picker=True, deg_emp=False, include_price=False, ignore_price_reward=True, ignore_invalid_penalty=True, ignore_overcharging_penalty=True, ignore_overloading_penalty=True, episode_length=36)

In [9]:
check_env(env)

Initial SoC changed from 0 to default value: 0.5
Invalid action, penalty given.
Timestep: 2020-01-02 10:00:00
SOC: [0.5], Time left: [0.] hours
Action taken: [-0.81903356]
Actual charging energy: 0 kWh
Charging cost/revenue: 0.0 €
SoH: [1.]
--------------------------
Reward signal: 0.0
---------


Invalid action, penalty given.
Timestep: 2020-01-02 10:15:00
SOC: [0.5], Time left: [0.] hours
Action taken: [-0.44374073]
Actual charging energy: 0 kWh
Charging cost/revenue: 0.0 €
SoH: [1.]
--------------------------
Reward signal: 0.0
---------


Invalid action, penalty given.
Timestep: 2020-01-02 10:30:00
SOC: [0.5], Time left: [0.] hours
Action taken: [0.528285]
Actual charging energy: 0 kWh
Charging cost/revenue: 0.0 €
SoH: [1.]
--------------------------
Reward signal: 0.0
---------


Invalid action, penalty given.
Timestep: 2020-01-02 10:45:00
SOC: [0.5], Time left: [0.] hours
Action taken: [-0.19870594]
Actual charging energy: 0 kWh
Charging cost/revenue: 0.0 €
SoH: [1.]
------------

In [10]:
n_actions = env.action_space.shape[-1]
param_noise = None
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5)*np.ones(n_actions))

In [11]:
model = TD3('MlpPolicy', env, verbose=1, tensorboard_log="./FleetRl_tensorboard_single/", learning_rate=0.001, learning_starts=10000, buffer_size=50000, batch_size=128)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
saving_interval = 20000
for i in range(1, 5):
    model.learn(total_timesteps=saving_interval, reset_num_timesteps=False, tb_log_name=f"TD3_{time_now}")
    model.save(f"{trained_agents_lmd}/{saving_interval * i}")

In [ ]:
del model
model = TD3.load(f"{trained_agents_lmd}/{saving_interval * 4}", env=env)
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

In [ ]:
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(24):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render()

Change: random time instead of always the same

In [ ]:
env = gym.make('FleetEnv-v0', schedule_name="lmd_sched_single.csv", building_name="load_lmd.csv", include_building=False, include_pv=False, static_time_picker=False, deg_emp=False, include_price=False, ignore_price_reward=True, ignore_invalid_penalty=True, ignore_overcharging_penalty=True, ignore_overloading_penalty=True, episode_length=36)

In [1]:
model = TD3.load(f"{trained_agents_lmd}/{saving_interval*4}", env=env)

In [ ]:
time_now = int(time.time())
comment = "step2"
trained_agents_lmd = f"./trained/last_mile_delivery/TD3-{time_now}-{comment}"
logs_lmd = f"./logs/last_mile_delivery/TD3-{time_now}-{comment}"

In [ ]:
saving_interval = 20000
for i in range(1, 5):
    model.learn(total_timesteps=saving_interval, reset_num_timesteps=False, tb_log_name=f"TD3_{time_now}")
    model.save(f"{trained_agents_lmd}/{saving_interval * i}")